In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import time
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import pickle
import os
from tqdm import tqdm

In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Input, BatchNormalization, Dropout, concatenate, PReLU, Flatten, Concatenate
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Reshape, Lambda, concatenate, dot, add, RepeatVector
from keras.layers import Dropout, GaussianDropout, multiply, SpatialDropout1D, BatchNormalization, subtract
from keras.utils import Sequence
from keras.callbacks import Callback
from keras import backend as K

Using TensorFlow backend.


In [3]:
invite_info = pd.read_table('../data/data_set_0926/invite_info_0926.txt',header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

In [4]:
data = pd.read_pickle('./data/data_all.pickle')

In [5]:
origin_feature   = ['用户二分类特征A', '用户二分类特征B', '用户二分类特征C','用户二分类特征D', '用户二分类特征E','用户的盐值分数']
lbl_enc_feat = ['性别_lbl_enc','访问频率_lbl_enc','用户分类特征A_lbl_enc', '用户分类特征B_lbl_enc', '用户分类特征C_lbl_enc', '用户分类特征D_lbl_enc','用户分类特征E_lbl_enc']
user_ques_svd    = ['问题绑定的话题ID_svd_{}'.format(i) for i in range(1,31)]
user_ques_watched_svd  = ['用户关注的话题_svd_{}'.format(i) for i in range(1,11)]
user_ques_fav_svd  = ['user_ques_fav_svd_{}'.format(i) for i in range(1,11)]
nn_stack = ['w2v_sum_nn']

test_feat = ['问题邀请用户_counts','用户被邀请问题_counts','用户的盐值分数_max',
                 '用户的盐值分数_min','用户的盐值分数_std','用户的盐值分数_mean','invite_answer_gap','邀请问题创建时间_gap',
                '用户关注的话题_len','用户感兴趣的话题_len','问题绑定的话题ID_len','prev_ans_ques_title_sim_min', 'prev_ans_ques_title_sim_max',
           'prev_ans_ques_title_sim_mean', 'prev_ans_ques_title_sim_std']
    
new_feat = ['用户ID_last_expo','用户ID_next_expo','问题ID_last_expo','问题ID_next_expo','问题创建时间_H','邀请创建时间_H']
    
new_feat_II = ['用户_问题IDs_svd_{}'.format(i) for i in range(1,11)]+['问题_用户IDs_svd_{}'.format(i) for i in range(1,11)]+['lstm_enc_feat']+\
     ['邀请创建时间_mean','邀请创建时间_max','邀请创建时间_min','邀请创建时间_std','问题创建时间_mean','问题创建时间_max','问题创建时间_min',
      '问题创建时间_std','用户ID曾经回答数','用户问题ID_count','用户问题话题相同个数','用户感兴趣问题话题相同个数']
      
new_feat_III = ['问题_用户感兴趣_topic_sim','问题_用户关注_topic_sim','用户关注_感兴趣_topic_sim']
    
new_feat_IV = ['回答是否被标优_count', '回答是否被推荐_count', '是否包含图片_count' ,
                   '是否包含视频_count','回答字数_mean' ,'点赞数_mean', '取赞数_mean' ,'评论数_mean' ,
                   '收藏数_mean', '感谢数_mean' ,'举报数_mean', '没有帮助数_mean' ,'反对数_mean']
    
new_feat_V = ['回答字数_sum' ,'点赞数_sum', '取赞数_sum' ,'评论数_sum' ,'收藏数_sum', '感谢数_sum' ,
                  '举报数_sum', '没有帮助数_sum' ,'反对数_sum']

new_feat_VI = ['问题标题_曾经回答_SW_sim','问题描述_曾经回答_SW_sim','问题标题_曾经回答_W_sim','问题描述_曾经回答_W_sim','times_mean',
                   'times_max','times_min','times_std','Hs_mean','Hs_max','Hs_min','Hs_std']
    
new_feat_VII = ['问题描述_曾经回答_Topic_sim','prev_topic_sims_min','prev_topic_sims_max','prev_topic_sims_mean','prev_topic_sims_std']
feat_I =['question_curr_expo','user_curr_expo']
feat_II = ['prev_ans_times_min_gap', 'prev_ans_times_mean_gap', 'qtime_std','qtime_mean', 'utime_std', 'utime_mean', 'iweek', 'qlast_itime_gap','qllast_itime_gap', 'qlllast_itime_gap', 'qnext_itime_gap','qnnext_itime_gap', 'qnnnext_itime_gap', 'ulast_itime_gap','ullast_itime_gap', 'ulllast_itime_gap', 'unext_itime_gap','unnext_itime_gap', 'unnnext_itime_gap']
new_feat_VIII = feat_I + feat_II 
ques_len_stat_feat = ['问题标题_len', '问题描述_len', '问题描述_len-问题标题_len', '问题标题_W_len', '问题描述_W_len','问题描述_len-问题标题_len_W', '问题标题和描述的交集个数', '问题标题和描述的交集个数_W', '问题标题和描述的交集个数/问题标题_len', '问题标题和描述的交集个数/问题描述_len', '问题标题和描述的交集个数_W/问题标题_len', '问题标题和描述的交集个数_W/问题描述_len', '编辑距离',
       '前一个词语是否相同', '前两个词语是否相同', '前三个词语是否相同', '前一个词语是否相同_W', '前两个词语是否相同_W','前三个词语是否相同_W', '第一个词语在标题里面出现位置', '第二个词语在标题里面出现位置', '第三个词语在标题里面出现位置','第一感兴趣在问题绑定话题里面出现位置', '第二感兴趣在问题绑定话题里面出现位置', '第三感兴趣在问题绑定话题里面出现位置', '问题话题编辑距离',
       '问题标题_len_mean', '问题描述_len_mean', '问题标题_W_len_mean', '问题描述_W_len_mean']    
#******* Feature sum***********#
feature  =    origin_feature + lbl_enc_feat + user_ques_svd + user_ques_watched_svd + user_ques_fav_svd + nn_stack +\
                   test_feat + new_feat + new_feat_II + new_feat_III + new_feat_IV + new_feat_V + new_feat_VI+\
                  new_feat_VII + new_feat_VIII + ques_len_stat_feat

print(len(feature))

211


In [6]:
cat_feat = ['用户二分类特征A', '用户二分类特征B', '用户二分类特征C','用户二分类特征D', '用户二分类特征E','性别_lbl_enc','访问频率_lbl_enc','用户分类特征A_lbl_enc', '用户分类特征B_lbl_enc', '用户分类特征C_lbl_enc', '用户分类特征D_lbl_enc','用户分类特征E_lbl_enc']
num_feat = [f for f in feature if f not in cat_feat]

In [7]:
for f in tqdm(cat_feat):
    data[f] = data[f].fillna('-1')
    data[f] = LabelEncoder().fit_transform(data[f])

100%|██████████| 12/12 [00:12<00:00,  1.08s/it]


In [8]:
for f in tqdm(num_feat):
    arr = data[f].values
    na_idx = np.isnan(arr)
    inf_idx = np.isinf(arr)
    arr[na_idx] = 0
    arr[inf_idx] = 0
    max_, min_ = arr.max(), arr.min()
    arr = (arr - min_) / (max_ - min_)
    arr[na_idx] = -0.1
    arr[inf_idx] = 1.1
    arr += 0.1
    data[f] = arr

100%|██████████| 199/199 [02:18<00:00,  1.43it/s]


In [9]:
len_train = len(invite_info)
train = data[:len_train]
test = data[len_train:]

In [10]:
single_embed_cnt = [(f, data[f].nunique()) for f in cat_feat]

In [12]:
single_embed_cnt

[('用户二分类特征A', 2),
 ('用户二分类特征B', 2),
 ('用户二分类特征C', 2),
 ('用户二分类特征D', 2),
 ('用户二分类特征E', 2),
 ('性别_lbl_enc', 3),
 ('访问频率_lbl_enc', 5),
 ('用户分类特征A_lbl_enc', 2315),
 ('用户分类特征B_lbl_enc', 254),
 ('用户分类特征C_lbl_enc', 402),
 ('用户分类特征D_lbl_enc', 1360),
 ('用户分类特征E_lbl_enc', 2)]

In [11]:
del data
gc.collect()

60

In [12]:
train_x = train[feature].reset_index(drop=True)
train_y = train['邀请是否被回答'].reset_index(drop=True)
test_x = test[feature].reset_index(drop=True)

In [13]:
print(test_x.shape)
print(train_x.shape)

(1141718, 211)
(9489162, 211)


In [14]:
del train, test
gc.collect()

60

In [15]:
def DNN(single_embed_cnt, inp_dense_dim, embed_size=32):
    
    # single embedding
    inp_single_embed = []
    out_single_embed = []
    out_lr = []
    for feat_name, inp_embed_dim in single_embed_cnt:
        inp = Input(shape=(1,))
        inp_single_embed.append(inp)
        x = Embedding(inp_embed_dim, embed_size)(inp)
        x = Flatten()(x)
        out_single_embed.append(x)
        # 
        x = Embedding(inp_embed_dim, 1)(inp)
        x = Flatten()(x)
        out_lr.append(Dense(1)(x))
    
    # dense
    inp_dense = Input(shape=(inp_dense_dim,))
    out_lr.append(Dense(1)(inp_dense))
    x = Dense(256)(inp_dense)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    out_dense = Dropout(0.2)(x)
    
    
    # concat
    conc = concatenate(out_single_embed + [out_dense])
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)
    conc = Dense(1)(conc)
    conc = concatenate(out_lr + [conc])
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)
    out = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp_single_embed+[inp_dense], outputs=out)
    
    return model

In [16]:
class DataSequence(Sequence):
    
    def __init__(self, x, y, single_embed_feat, dense_feat, batch_size=128):
        self.x = x
        self.y = y
        self.single_embed_feat = single_embed_feat
        self.dense_feat = dense_feat
        self.batch_size = batch_size
#         self.x_single_embed = [x[f].values for f in single_embed_feat]
#         self.x_dense = x[dense_feat].values
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch = self.x.iloc[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_x = [batch[f].values for f in self.single_embed_feat] + [batch[f].values for f in self.dense_feat]
        batch_x = [batch[f].values for f in self.single_embed_feat] + [batch[self.dense_feat].values]
#         batch_x = [xf[idx * self.batch_size:(idx + 1) * self.batch_size] for xf in self.x_single_embed] \
#                 + [self.x_dense[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [17]:
class MetricsCallback(Callback):
    def __init__(self, x_trn, y_trn, x_val, y_val, cat_feat, num_feat, batch_size=50000, save_name='weight.h5'):
        self.trn_generator = DataSequence(x_trn, y_trn, cat_feat, num_feat, BATCH_SIZE)
        self.val_generator = DataSequence(x_val, y_val, cat_feat, num_feat, BATCH_SIZE)
        self.y_trn = y_trn
        self.y_val = y_val
        self.save_name = save_name
        self.best_score = 0.5

    def on_epoch_end(self, epoch, logs={}):
        
        y_pred = self.model.predict_generator(self.trn_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc = roc_auc_score(self.y_trn, y_pred)
        y_pred_val = self.model.predict_generator(self.val_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        
        if roc_val > self.best_score:
            self.best_score = roc_val
            self.model.save_weights(os.path.join(WEIGHT_PATH, self.save_name))

        return

In [18]:
import time
tic = time.time()
BATCH_SIZE = 10240
WEIGHT_PATH = './models/'
test_pred = np.zeros((len(test_x), 1))

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('-'*100)
    print('Fold %d' % i)
    trn_x, trn_y = train_x.iloc[tr_idx], train_y[tr_idx]
    val_x, val_y = train_x.iloc[va_idx], train_y[va_idx]
    
    K.clear_session()
    model = DNN(single_embed_cnt, len(num_feat), embed_size=32)
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    trn_generator = DataSequence(trn_x, trn_y, cat_feat, num_feat, batch_size=BATCH_SIZE)
    val_generator = DataSequence(val_x, val_y, cat_feat, num_feat, batch_size=BATCH_SIZE)
    test_generator = DataSequence(test_x, np.zeros(len(test_x)), cat_feat, num_feat, batch_size=BATCH_SIZE)
    history = model.fit_generator(generator=trn_generator, 
                        epochs=30, 
                        verbose=1, 
                        callbacks=[MetricsCallback(trn_x, trn_y, val_x, val_y, cat_feat, num_feat, 
                                                   batch_size=BATCH_SIZE*4,
                                                   save_name='main_dnn_f%d.h5' % i)], 
                        max_queue_size=10, 
                        workers=1, 
                        use_multiprocessing=False)
    
    model.load_weights(os.path.join(WEIGHT_PATH, 'main_dnn_f%d.h5' % i))
    
    test_pred += model.predict_generator(test_generator) / 5
    
    del trn_x, trn_y, val_x, val_y
    gc.collect()
print("Used time: %d s" % (time.time()-tic))

----------------------------------------------------------------------------------------------------
Fold 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
742/742 [==============================] - 46s 63ms/step - loss: 0.4224
roc-auc: 0.7867 - roc-auc_val: 0.7862                                                                                                    
Epoch 2/30
742/742 [==============================] - 41s 55ms/step - loss: 0.3654
roc-auc: 0.8313 - roc-auc_val: 0.8298                                                                                                    
Epoch 3/30
742/742 [==============================] - 41s 56ms/step - loss: 0.3570
roc-auc: 0.8485 - roc-auc_val: 0.8473                                                                                                    
Epoch 4/30
742/742 [==============================] - 43s 58ms/step - loss: 0.3516
roc-auc: 0.8451 - roc-auc_

742/742 [==============================] - 56s 76ms/step - loss: 0.3485
roc-auc: 0.824 - roc-auc_val: 0.8223                                                                                                    
Epoch 6/30
742/742 [==============================] - 48s 65ms/step - loss: 0.3454
roc-auc: 0.8476 - roc-auc_val: 0.8463                                                                                                    
Epoch 7/30
742/742 [==============================] - 45s 61ms/step - loss: 0.3428
roc-auc: 0.8598 - roc-auc_val: 0.8578                                                                                                    
Epoch 8/30
742/742 [==============================] - 46s 62ms/step - loss: 0.3407
roc-auc: 0.8489 - roc-auc_val: 0.8459                                                                                                    
Epoch 9/30
742/742 [==============================] - 45s 61ms/step - loss: 0.3390
roc-auc: 0.8463 - roc-auc_val: 0.8435        

742/742 [==============================] - 40s 54ms/step - loss: 0.3347
roc-auc: 0.8718 - roc-auc_val: 0.8684                                                                                                    
Epoch 13/30
742/742 [==============================] - 43s 58ms/step - loss: 0.3338
roc-auc: 0.8498 - roc-auc_val: 0.8465                                                                                                    
Epoch 14/30
742/742 [==============================] - 41s 55ms/step - loss: 0.3331
roc-auc: 0.8469 - roc-auc_val: 0.8438                                                                                                    
Epoch 15/30
742/742 [==============================] - 41s 55ms/step - loss: 0.3324
roc-auc: 0.8672 - roc-auc_val: 0.8628                                                                                                    
Epoch 16/30
742/742 [==============================] - 43s 57ms/step - loss: 0.3320
roc-auc: 0.883 - roc-auc_val: 0.8785    

KeyboardInterrupt: 

In [19]:
invite_info_evaluate = pd.read_table('../data/data_set_0926/invite_info_evaluate_2_0926.txt',header=None)
invite_info_evaluate.columns =  ['问题ID','用户ID','邀请创建时间']
result = invite_info_evaluate[['问题ID','用户ID','邀请创建时间']]
result['result'] = test_pred
result.to_csv('./out/result_dnn.txt', sep='\t', index=False, header=False)